# Titanic Survivor Prediction Model
This notebook trains a VotingClassifier ensemble using 5 scikit-learn models to predict Titanic survival.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import joblib

In [ ]:
# Load Data
data_path = '../data/titanic/train.csv'
df = pd.read_csv(data_path)
df.head()

In [ ]:
# Feature Selection
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = df[features]
y = df['Survived']

# Preprocessing Setup
numeric_features = ['Age', 'SibSp', 'Parch', 'Fare']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_features = ['Pclass', 'Sex', 'Embarked']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
# Model Definitions
clf1 = LogisticRegression(random_state=42)
clf2 = DecisionTreeClassifier(random_state=42)
clf3 = RandomForestClassifier(random_state=42)
clf4 = SVC(probability=True, random_state=42)
clf5 = KNeighborsClassifier()

# Ensemble (Voting Classifier)
eclf = VotingClassifier(
    estimators=[('lr', clf1), ('dt', clf2), ('rf', clf3), ('svc', clf4), ('knn', clf5)],
    voting='soft'
)

# Pipeline
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', eclf)])

In [ ]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train
model_pipeline.fit(X_train, y_train)

In [ ]:
# Evaluate
print(f"Train Score: {model_pipeline.score(X_train, y_train):.4f}")
print(f"Test Score: {model_pipeline.score(X_test, y_test):.4f}")

In [ ]:
# Save Model
joblib.dump(model_pipeline, 'model.pkl')
print("Model saved to model.pkl")